In [ ]:
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import FuncTickFormatter, ColumnDataSource, CategoricalColorMapper
from bokeh.themes.theme import Theme
from bokeh.transform import jitter
from bokeh.io import show

import holoviews as hv

import reloadr
from aleph_scoring.scoring import compute_ccn_score

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap


output_notebook()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
store = pd.HDFStore('metrics.hdf5', mode='r')

In [ ]:
compute_ccn_score = reloadr.reloadr(compute_ccn_score)

compute_ccn_score._reload()
ccn_metrics = compute_ccn_score(store["core_channel_nodes"])

In [ ]:
def box_plt(dataframe, metric):

    # generate some synthetic time series for six different categories
    df = dataframe[["node_id", metric]]
    cats = list(dataframe["node_id"].unique())
    #df['short_id'] = df['node_id'].str[-10:]


    # find the quartiles and IQR for each category
    groups = df.groupby('node_id')
    q1 = groups.quantile(q=0.25)
    q2 = groups.quantile(q=0.5)
    q3 = groups.quantile(q=0.75)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr

    # find the outliers for each category
    def outliers(group):
        cat = group.name
        return group[(group[metric]  > upper.loc[cat][metric]) | (group[metric] < lower.loc[cat][metric])][metric]
    out = groups.apply(outliers).dropna()

    # prepare outlier data for plotting, we need coordinates for every outlier.
    if not out.empty:
        outx = list(out.index.get_level_values(0))
        outy = list(out.values)

    p = figure(background_fill_color="#efefef", x_range=cats, sizing_mode='stretch_width')

    # if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
    qmin = groups.quantile(q=0.00)
    qmax = groups.quantile(q=1.00)
    upper[metric] = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,metric]),upper[metric])]
    lower[metric] = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,metric]),lower[metric])]

    # stems
    p.segment(cats, upper[metric], cats, q3[metric], line_color="black")
    p.segment(cats, lower[metric], cats, q1[metric], line_color="black")

    # boxes
    p.vbar(cats, 0.7, q2[metric], q3[metric], fill_color="#E08E79", line_color="black")
    p.vbar(cats, 0.7, q1[metric], q2[metric], fill_color="#3B8686", line_color="black")

    # whiskers (almost-0 height rects simpler than segments)
    p.rect(cats, lower[metric], 0.2, 0.01, line_color="black")
    p.rect(cats, upper[metric], 0.2, 0.01, line_color="black")

    # outliers
    if not out.empty:
        p.circle(outx, outy, size=6, color="#F38630", fill_alpha=0.6)
    
    p.xaxis.axis_label_text_font = "FreeMono"
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = "white"
    p.grid.grid_line_width = 2
    p.xaxis.major_label_text_font_size="12px"
    p.xaxis.major_label_orientation = "vertical"
    
    p.xaxis.formatter = FuncTickFormatter(code="""
        return tick.slice(-10)
    """)

    show(p)

In [ ]:

box_plt(ccn_metrics, "score_base_latency")

In [ ]:
data = ccn_metrics[["node_id", "score_base_latency", "score_metrics_latency", "score_aggregate_latency", "score_file_download_latency", "score_pending_messages", "score_eth_height_remaining", "score"]].reset_index()
data["node_id"] = data["node_id"].str[-10:]
  
data.sort_values(by=["node_id"])


In [ ]:
def box_plt(dataframe, metric):
    hv.extension('bokeh')


    theme = Theme(
        json={
        'attrs' : {
            'Axis': {
                "major_label_text_font": "FreeMono",
                "axis_label_text_font": "FreeMono",

            }
        }
    })
    hv.renderer('bokeh').theme = theme

    data = ccn_metrics
    data["node_id"] = data["node_id"].str[-10:]
    data = data.sort_values(by=['node_id'], ascending=False)

    title = "MPG by Cylinders and Data Source, Colored by Cylinders"
    boxwhisker = hv.BoxWhisker(data, "node_id", "score_base_latency", label="node_id")
    boxwhisker.opts(width=900, height=2500, title=metric, invert_axes=True)
    
    return boxwhisker


In [ ]:
box_plt(ccn_metrics, "score_base_latency")

In [ ]:


def scatter_plt(dataframe, metric):

    data = dataframe.sort_values(by="node_id", ascending=False)
    df = pd.DataFrame(data=data["node_id"].sort_values(ascending=False).unique())
    df.rename(columns={0: 'node_id'}, inplace=True)
    df['color'] = np.where(df.index % 2 != 0, 'B', 'R')
    data["node_id"] = data["node_id"].str[-10:]
    data = data.merge(df, on="node_id", how="left")

    node_id = data["node_id"].unique()

    source = ColumnDataSource(data)

    color_mapper = CategoricalColorMapper(factors=['B', 'R'], palette=["#6383EA", "#A9BAF3"])

    p = figure(height=2500, y_range=node_id, width=950,
               title=metric)
    p.circle(x=metric, y=jitter('node_id', width=0.6, range=p.y_range),color={'field': 'color', 'transform': color_mapper}, source=source, alpha=1)

    p.ygrid.grid_line_color = None

    show(p)

In [ ]:
scatter_plt(ccn_metrics, "score_base_latency")

In [ ]:
hv.extension('bokeh')


viridis = cm.get_cmap('coolwarm_r', 256)
newcolors = viridis(np.linspace(0, 1, 256))
pink = np.array([248/256, 24/256, 148/256, 1])
newcolors[:25, :] = pink
newcmp = ListedColormap(newcolors)

x = "coolwarm_r"
data = ccn_metrics[["node_id", "score_base_latency", "score_metrics_latency", "score_aggregate_latency", "score_file_download_latency", "score_pending_messages", "score_eth_height_remaining", "score"]].groupby("node_id").mean().reset_index().round(2)
data["node_id"] = data["node_id"].str[-10:]
data = data.melt(id_vars=["node_id"], 
        var_name="score", 
        value_name="value").sort_values(by=['node_id'],  ascending=True)

hm = hv.HeatMap(data, kdims = ['score', 'node_id']
                , vdims =['value']).sort()

# Create non linear colormap
cmap = LinearSegmentedColormap.from_list('custom blue', 
                                             [(0,    '#CC3F39'),
                                              (0.50, '#CC3F39'),
                                              (0.80, '#FFFFFF'),
                                              (0.95, '#6383EA'),
                                              (1,    '#6383EA')], N=256)

hm.opts(xrotation=45,tools=['hover'], colorbar=True, colorbar_position="top", xaxis="top", 
        width=950, height=1500, cmap=cmap, invert_yaxis=True,
       fontsize={ 'title': 15, 'labels': 5, 'xticks': 10, 'yticks': 10,})

hm * hv.Labels(hm).opts(text_font_size='10pt')
